In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from dataset.dataset_loader import SNDataset, myNormalize, myToTensor
from torchvision import transforms

In [2]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
mynorm = myNormalize()
my_to_tensor = myToTensor()
transform = transforms.Compose([mynorm, my_to_tensor])

In [4]:
train_ds = SNDataset('D:\python\SoilNet\dataset\l8_images\\train\\','D:\python\SoilNet\dataset\LUCAS_2015_Germany_all.csv', transform=transform)
test_ds = SNDataset('D:\python\SoilNet\dataset\l8_images\\test\\','D:\python\SoilNet\dataset\LUCAS_2015_Germany_all.csv', transform=transform)

In [5]:
# CONFIG
NUM_WORKERS = 2
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 2

In [6]:
train_dl = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dl = DataLoader(test_ds, batch_size=TEST_BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

In [7]:
# Get image and label from custom DataLoader
img_custom, label_custom = next(iter(train_dl))

print(f"Image shape: {img_custom.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label_custom.shape}")

Image shape: torch.Size([8, 12, 64, 64]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([8])


In [8]:
from soilnet.soil_net import SoilNetFC

In [9]:
model = SoilNetFC(device=device).to(device)
img_gpu = img_custom.to(device)
print(img_gpu.device)
y = model(img_gpu)
print(y.detach().shape)


cuda:0


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)

In [ ]:
for param in model.parameters():
  print(param.name, param.shape, param.device)
print("---------------------")
for buffer in model.buffers():
  print(buffer.name, buffer.shape, buffer.device)
    

None torch.Size([96, 1, 3, 3]) cuda:0
None torch.Size([96]) cuda:0
None torch.Size([12, 96]) cuda:0
None torch.Size([96, 12]) cuda:0
None torch.Size([64, 96, 4, 4]) cuda:0
None torch.Size([64]) cuda:0
None torch.Size([128, 64, 4, 4]) cuda:0
None torch.Size([128]) cuda:0
None torch.Size([256, 128, 4, 4]) cuda:0
None torch.Size([256]) cuda:0
None torch.Size([512, 256, 4, 4]) cuda:0
None torch.Size([512]) cuda:0
None torch.Size([1024, 512, 4, 4]) cuda:0
None torch.Size([1024]) cuda:0
None torch.Size([1024, 1024, 2, 2]) cuda:0
None torch.Size([1024]) cuda:0
None torch.Size([128, 128]) cuda:0
None torch.Size([128]) cuda:0
None torch.Size([1, 128]) cuda:0
None torch.Size([1]) cuda:0
---------------------


In [ ]:
def train_step(model:nn.Module, data_loader:DataLoader, loss_fn:nn.Module, optimizer:torch.optim.Optimizer):
    size = len(data_loader.dataset)
    model.train()
    for batch, (X, y) in enumerate(data_loader):
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")